In [6]:
import os
import time
from datetime import datetime
from dotenv import load_dotenv

import googlemaps
import gmplot

import pandas as pd

from io import StringIO
import uuid
import ast

import requests
from geopy.distance import geodesic

In [7]:
load_dotenv()
API_KEY = os.getenv('GOOGLE_MAPS_API_KEY_3')

gmaps = googlemaps.Client(key=API_KEY)

In [11]:
def get_route_directions(start, end):
    directions_result = gmaps.directions(start, end, mode="driving", alternatives=True)
    routes = []
    for route in directions_result:
        polyline = route['overview_polyline']['points']
        decoded_points = googlemaps.convert.decode_polyline(polyline)
        routes.append({
            "route": decoded_points,
            "summary": route["summary"],
            "legs": route["legs"]
        })
    return routes

def check_route_crosses_avoid_list(route, avoid_list, distance_threshold=0.05):
    for point in route:
        for (avoid_start, avoid_end) in avoid_list:
            if check_point_near_segment(point, avoid_start, avoid_end, distance_threshold):
                return True
    return False

def check_point_near_segment(point, start, end, distance_threshold):
    distance_start = geodesic((point['lat'], point['lng']), (start[0], start[1])).km
    distance_end = geodesic((point['lat'], point['lng']), (end[0], end[1])).km
    if distance_start <= distance_threshold or distance_end <= distance_threshold:
        return True
    return False

def find_best_route_avoiding_streets(start, end, avoid_list):
    all_routes = get_route_directions(start, end)
    avoided_streets = []
    for route_data in all_routes:
        route = route_data["route"]
        avoided_any = False
        for (avoid_start, avoid_end) in avoid_list:
            if not check_route_crosses_avoid_list(route, [(avoid_start, avoid_end)]):
                avoided_any = True
                avoided_streets.append((avoid_start, avoid_end))
        if avoided_any:
            return route_data, avoided_streets
    return None, avoided_streets

def print_avoid_list(avoid_list):
    for idx, (start, end) in enumerate(avoid_list, 1):
        print(f"{idx}. From {start} to {end}")

def print_route_directions(route_legs):
    for leg in route_legs:
        print(f"Route from {leg['start_address']} to {leg['end_address']}")
        print(f"Distance: {leg['distance']['text']}, Duration: {leg['duration']['text']}")
        for step in leg['steps']:
            print(f"- {step['html_instructions']} ({step['distance']['text']})")
        print("\n")

start_point = (24.760322, 46.732212)
end_point = (24.768003, 46.627667) 

avoid_list = [
    ((24.7970264, 46.719939), (24.7388275, 46.59441409999999)),
    ((24.954535, 47.0142416), (24.7258606, 46.583506)),
    ((24.796827, 46.5643251), (24.7089077, 46.6195443)),
    ((24.9229714, 46.7204701), (24.796827, 46.5643251)),
    ((24.796827, 46.5643251), (24.6575642, 46.5630617)),
    ((24.7575596, 46.6895021), (24.70444, 46.6237931)),
]

print_avoid_list(avoid_list)

best_route, avoided_streets = find_best_route_avoiding_streets(start_point, end_point, avoid_list)

if best_route:
    print(f"\nBest route summary: {best_route['summary']}")
    print("Avoided streets:")
    for street in avoided_streets:
        print(street)
    print_route_directions(best_route["legs"])
else:
    print("No valid route found that avoids the restricted streets.")

1. From (24.7970264, 46.719939) to (24.7388275, 46.59441409999999)
2. From (24.954535, 47.0142416) to (24.7258606, 46.583506)
3. From (24.796827, 46.5643251) to (24.7089077, 46.6195443)
4. From (24.9229714, 46.7204701) to (24.796827, 46.5643251)
5. From (24.796827, 46.5643251) to (24.6575642, 46.5630617)
6. From (24.7575596, 46.6895021) to (24.70444, 46.6237931)

Best route summary: King Abdullah Rd
Avoided streets:
((24.7970264, 46.719939), (24.7388275, 46.59441409999999))
((24.954535, 47.0142416), (24.7258606, 46.583506))
((24.796827, 46.5643251), (24.7089077, 46.6195443))
((24.9229714, 46.7204701), (24.796827, 46.5643251))
((24.796827, 46.5643251), (24.6575642, 46.5630617))
((24.7575596, 46.6895021), (24.70444, 46.6237931))
Route from 6944 Al Bunyan, 3974, Al Mughrizat, Riyadh 12482, Saudi Arabia to حي, RRAA2697, 2697 Al Authymah Aljadeeda, 7603, Al Aqiq, Riyadh 13511, Saudi Arabia
Distance: 16.4 km, Duration: 18 mins
- Head <b>southeast</b> on <b>Al Bounian</b>/<wbr/><b>Jarir Ibn A

In [23]:
import googlemaps
from polyline import decode


# Fetch directions from Google Maps API
def get_directions(start, end, alternatives=False):
    directions_result = gmaps.directions(start, end, mode="driving", alternatives=alternatives)
    if directions_result:
        return directions_result
    return None

# Decode a Google Maps encoded polyline to latitude and longitude points
def decode_polyline_to_points(polyline):
    return decode(polyline)

# Check if two routes intersect based on their polylines
def do_routes_intersect(route_points_a, route_points_b):
    # Compare points in both routes to see if any points are near each other
    for point_a in route_points_a:
        for point_b in route_points_b:
            if point_a == point_b:  # Check if the routes share a point
                return True  # Routes intersect at some point
    return False  # No intersection

# Find a route that avoids all segments in avoid_list
def find_route_avoiding_segments(start, end, avoid_list):
    directions_a_b = get_directions(start, end, alternatives=True)

    if not directions_a_b:
        return None, None

    # Check each route from A to B
    for route in directions_a_b:
        route_a_b_points = decode_polyline_to_points(route['overview_polyline']['points'])

        # Check this route against all avoidable segments in avoid_list
        avoid_crossing = False
        for avoid_start, avoid_end in avoid_list:
            directions_c_d = get_directions(avoid_start, avoid_end, alternatives=False)
            if directions_c_d:
                route_c_d_points = decode_polyline_to_points(directions_c_d[0]['overview_polyline']['points'])
                if do_routes_intersect(route_a_b_points, route_c_d_points):
                    avoid_crossing = True
                    break  # This route crosses an avoidable segment, so skip it

        if not avoid_crossing:
            # Found a valid route that doesn't cross any avoidable segments
            return route['legs'][0]

    return None

# Generate Google Maps Directions link for a valid route
def generate_google_maps_link(route):
    start_location = route['start_location']
    end_location = route['end_location']

    gmaps_link = f"https://www.google.com/maps/dir/?api=1&origin={start_location['lat']},{start_location['lng']}&destination={end_location['lat']},{end_location['lng']}&travelmode=driving"

    return gmaps_link

# Start and end points for the route A -> B
start_point = (24.760322, 46.732212)  # Point A
end_point = (24.743762, 46.652182)    # Point B

# List of avoidable segments (C -> D pairs)
avoid_list = [
    [(24.7970264, 46.719939), (24.7388275, 46.59441409999999)],
    [(24.954535, 47.0142416), (24.7258606, 46.583506)],
    [(24.796827, 46.5643251), (24.7089077, 46.6195443)],
    [(24.9229714, 46.7204701), (24.796827, 46.5643251)],
    [(24.796827, 46.5643251), (24.6575642, 46.5630617)],
    [(24.7575596, 46.6895021), (24.70444, 46.6237931)],
]

# Find a route from A -> B that avoids crossing any segment in avoid_list
route = find_route_avoiding_segments(start_point, end_point, avoid_list)

if route:
    gmaps_link = generate_google_maps_link(route)
    print("Google Maps Directions Link:", gmaps_link)
else:
    print("No valid route found that avoids crossing the segments in avoid_list.")


Google Maps Directions Link: https://www.google.com/maps/dir/?api=1&origin=24.7603566,46.7322952&destination=24.7437281,46.6521032&travelmode=driving


In [18]:
# import googlemaps
# from polyline import decode

# # Fetch directions from Google Maps API
# def get_directions(start, end, alternatives=False):
#     directions_result = gmaps.directions(start, end, mode="driving", alternatives=alternatives)
#     if directions_result:
#         return directions_result
#     return None

# # Decode a Google Maps encoded polyline to latitude and longitude points
# def decode_polyline_to_points(polyline):
#     return decode(polyline)

# # Check if two routes intersect based on their polylines
# def do_routes_intersect(route_points_a, route_points_b):
#     # Compare points in both routes to see if any points are near each other
#     for point_a in route_points_a:
#         for point_b in route_points_b:
#             if point_a == point_b:  # Check if the routes share a point
#                 return True  # Routes intersect at some point
#     return False  # No intersection

# # Find a route that avoids the C -> D segment by checking if A -> B crosses C -> D
# def find_route_avoiding_segment(start, end, avoid_start, avoid_end):
#     directions_a_b = get_directions(start, end, alternatives=True)
#     directions_c_d = get_directions(avoid_start, avoid_end, alternatives=False)

#     if not directions_a_b or not directions_c_d:
#         return None, None

#     # Decode the avoidable segment (C -> D) polyline to points
#     route_c_d_points = decode_polyline_to_points(directions_c_d[0]['overview_polyline']['points'])

#     for route in directions_a_b:
#         # Decode the A -> B route polyline to points
#         route_a_b_points = decode_polyline_to_points(route['overview_polyline']['points'])

#         # Check if the A -> B route intersects the C -> D segment
#         if not do_routes_intersect(route_a_b_points, route_c_d_points):
#             return route['legs'][0]  # Return this route as it avoids the segment C -> D

#     return None

# # Generate Google Maps Directions link for a valid route
# def generate_google_maps_link(route):
#     start_location = route['start_location']
#     end_location = route['end_location']

#     gmaps_link = f"https://www.google.com/maps/dir/?api=1&origin={start_location['lat']},{start_location['lng']}&destination={end_location['lat']},{end_location['lng']}&travelmode=driving"

#     return gmaps_link

# # Start and end points for the route A -> B
# start_point = (24.760322, 46.732212)  # Point A
# end_point = (24.768003, 46.627667)    # Point B

# # Avoidable segment C -> D
# avoid_start = (24.7575596, 46.6895021)  # Point C
# avoid_end = (24.70444, 46.6237931)      # Point D

# # Find a route from A -> B that avoids crossing C -> D
# route = find_route_avoiding_segment(start_point, end_point, avoid_start, avoid_end)

# if route:
#     gmaps_link = generate_google_maps_link(route)
#     print("Google Maps Directions Link:", gmaps_link)
# else:
#     print("No valid route found that avoids crossing the C -> D segment.")


Google Maps Directions Link: https://www.google.com/maps/dir/?api=1&origin=24.7603566,46.7322952&destination=24.768036,46.6276509&travelmode=driving
